In [41]:
%load_ext autoreload
%autoreload 2

# general
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import matplotlib.cm as cm
import seaborn as sns

import scipy.io as io
import re
from scipy.signal import resample as scipy_resample
from functools import partial
from tqdm import tqdm

from ephyspy.features import *  # contains available the features
from ephyspy.allen_sdk import ephys_extractor
from ephyspy.sweeps import EphysSweepSet, EphysSweep

import jupyter_black

jupyter_black.load()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [42]:
from pandas import Series, DataFrame
from scipy.signal import filtfilt, bessel, iirnotch
from typing import Any


def filter_bad_sweeps(
    df: DataFrame,
    filt_df: DataFrame,
    align_by: str or Tuple[str, str],
    filter_by: str or Tuple[str, str],
) -> DataFrame:
    """Remove single sweeps (npoints,t,Ut,It) in dataframe as specified in another.

    Align two dataframes by a common column (does not neccesarily have to have
    the same name, only the same data) and merge them. Then enforce check which
    sweeps need to be excluded. Specified as a comma seperated string.

    Args:
        df: Dataframe to be filtered.
        filt_df: Dataframe for filtering.
        align_by: the same or two different columns that are used to align the
            two different dataframes.
        filter_by: columns to equate. see b)

    Returns:
        pd.Series with True where condition is met.
    """
    align_by = (align_by, align_by) if isinstance(align_by, str) else align_by
    filter_by = (filter_by, filter_by) if isinstance(filter_by, str) else filter_by
    orig_cols = df.columns

    if align_by[0] != align_by[1]:
        filt_df.rename({align_by[1]: align_by[0]}, inplace=True)
    df = pd.merge(df, filt_df, on=align_by[0], how="left")

    def clean_It(ts: ndarray, It: Callable, idxs2rm: List) -> Callable:
        """Remove stimuli corresponding to bad sweeps identified by idxs2rm."""
        if isinstance(ts, ndarray):
            ts = np.vstack([ts] + [ts[0]] * len(idxs2rm))
        return np.delete(It(ts), idxs2rm, axis=0)

    def sweep_filter(
        df: DataFrame, filter_by: str or Tuple[str, str]
    ) -> Tuple[ndarray, ndarray, ndarray, ndarray]:
        if not isinstance(df[filter_by], str):  # catches nans
            return df.npoints, df.t, df.Ut, df.It
        else:
            rm = [int(x) - 1 for x in df[filter_by].split(",")]
            keep = [x for x in range(df.Ut.shape[0]) if x not in rm]
            return (
                df.npoints[keep],
                df.t[keep],
                df.Ut[keep],
                partial(clean_It, It=df.It, idxs2rm=rm),
            )

    if not df.empty:
        df[["npoints", "t", "Ut", "It"]] = df.apply(
            lambda x: sweep_filter(x, filter_by[0]), axis=1, result_type="expand"
        )
        df = df[df[filter_by[0]] != "all"].reset_index()[orig_cols]

    return df


def trange2t(t: ndarray) -> ndarray:
    """Transform array with tranges to time array.

    Args:
        t: time ranges in the format t=[[t0,tend,dt],...]

    Return:
        time array of form t=[[t0,t1,...,tN],...].
    """
    assert t.shape[1] == 3, "trange needs to have format: t=[[t0,tend,dt],...]"
    t = t.copy()
    if np.any(np.diff(t[:, 1])) > 0:  # catch different len tranges of ramps
        t[:, 1] = t[-1, 1]
    return np.vstack([np.arange(*ts) for ts in t])


def mat2dicts(mat: Dict) -> List[Dict]:
    """Takes io.loadmat outputs for ephys data and does some preprocessing.

    Args:
        mat: output dicts of io.loadmat with squeeze_me=True,
            chars_as_strings=True, mat_dtype=False, simplify_cells=True.

    Returns:
        stack of preprocessed dictionaries."""
    dcts = []
    num_recs = len(re.findall("chan[1-9]+", ",".join(mat.keys())))
    for n in range(num_recs):
        head = mat[f"head{n+1}"]
        chan = mat[f"chan{n+1}"]

        # manual refmt and clean
        chan["adc"] = chan["adc"].T.astype(float)
        chan["adc"] = (
            chan["adc"] if chan["adc"].ndim > 1 else chan["adc"].reshape(1, -1)
        )
        chan["tim"] = chan["tim"].astype(float)
        chan.pop("mrk")

        fname = head.pop("source").pop("name")  # remove source info
        head["cell"] = re.findall(r"[^\/\\]+(?=\.)", fname)[0]  # filename -> cellname
        keys2pop = [
            [
                "tim",
                "Channeltype",
                "Patch",
                "channeltype",
                "channeltypeFcn",
                "classifier",
                "comment",
                "embeddeddata",
                "markerclass",
                "Environment",
                "mapstructure",
            ],
            ["Func", "Labels", "MultiInterval", "Multiplex", "TargetClass"],
        ]
        for key in keys2pop[0]:
            head.pop(key)
        for key in keys2pop[1]:
            head["adc"].pop(key)
        head.update(head.pop("adc"))
        head["label"] = head.pop("Group")["Label"]
        head["Npoints"] = (
            np.array([head["Npoints"]])
            if not isinstance(head["Npoints"], np.ndarray)
            else head["Npoints"]
        )

        head.update(chan)
        dcts.append(head)

    return dcts


def t2trange(t: ndarray) -> ndarray:
    """Transform time array to array with trange params.

    Args:
        t: time array of form t=[[t0,t1,...,tN],...].

    Return:
        time ranges in the format t=[[t0,tend,dt],...]
    """

    def ensure_correct_t_len(t):
        t0 = t[:, 0]
        tN = t[:, -1]
        dt = t[:, 1] - t0

        l_b4 = t.shape[1]
        l_after = len(np.arange(t0[0], tN[0], dt[0]))
        tN -= (l_after - l_b4) * dt
        return t0, tN, dt

    assert t.shape[1] != 3, "t needs to have format: t=[[t0,t1,...,tN],...]"
    t0, tN, dt = ensure_correct_t_len(t)
    return np.vstack([t0, tN, dt]).T


def get_regex_filter(col: str, regex: str, case_sensitive=False) -> Callable:
    """return filter for regexs in particular column of a dataframe.

    If regex is found, the value at the index is True, else False.

    Args:
        df: Dataframe for filtering.
        col: col name to filter.
        regex: regular expression that is matched for values in the column.

    Returns:
        Callable that takes df and returns Series that contains True/False if regex is matched in col.
    """
    match_regex = lambda x: x.str.contains(regex, regex=True)
    if case_sensitive:
        return lambda df: match_regex(df[col])
    else:
        return lambda df: match_regex(df[col].str.lower())


def import_mats(fpaths: List[str], filter_func: Optional[Callable] = None) -> DataFrame:
    r"""Import and reformat .mat files with electrophysiological data.

    Imports all files in given in fpaths. Each .mat file must at least contain
    keys 'chan1' & 'head1' to be amenable to this function.

    The information that is contained within 'head[i]' and 'chan[i]' is parsed
    and a dataframe is constructed that contains the header and data for each
    recording.

    This includes (pruned to contain only useful entries):
    cell: filename / name of the cell
    channel: channel of the recording
    label: type of recording (fp = firing pattern)
    dc: offset current
    npoints: number of timepoints
    t: timepoints
    Ut: voltage traces for different stimuli
    It: different stimuli

    The function takes care of rescaling the data in t, U and matches the stimuli
    to the voltage traces (assuming Ut roughly flat -> I=0, and increasing/decreasing
    I in 20pA steps around that). It also selects only traces that have labels
    containing 'fp', 'sAP' or 'ramp'.

    To save memory the output is returned as:
    t = [[t0,tend,dt],...] assuming equal spacing of timebins. (num_sweeps, 3)
    Ut with shape (num_sweeps, $N_t$)
    It as function that returns array of shape (num_sweeps, $N_t$) for every t
    Instead of as:
    t = (num_sweeps,$N_t$)
    Ut = (num_sweeps, $N_t$)
    It = (num_sweeps, $N_t$)
    Then `unzip` can be used to obtain arrays of the same shapes.

    Args:
        fpaths: list of paths to matfiles to import.
        filter_func: applies filter to imported mats. Can save memory to get rid
            of unwanted recordings already during import.

    Returns:
        header_df: dataframe containing header information and data for the
            electrophysiological recordings.
    """
    dfs = []
    for fpath in tqdm(fpaths, desc="importing"):
        mat = io.loadmat(
            fpath,
            squeeze_me=True,
            chars_as_strings=True,
            mat_dtype=False,
            simplify_cells=True,
        )
        dcts = mat2dicts(mat)
        df = pd.DataFrame(dcts)

        # pre-filter (i.e. remove Imons, EPSPs, EPSCs) -> more memory efficient in for loop, faster outside!
        unused_data_filter = get_regex_filter("label", "epsp|epsc|spont")
        filtered_df = filter_df(df, unused_data_filter(df), True)
        if filter_func is not None:
            pre_filter = (
                (lambda df: pd.Series([False] * len(df)))
                if filter_func is None
                else filter_func
            )
            filtered_df = filter_df(filtered_df, pre_filter)

        dfs.append(filtered_df)

    df = pd.concat(dfs).reset_index(drop=True)

    enough_sweeps = df["adc"].apply(lambda x: x.shape[0] > 2)
    df = filter_df(df, enough_sweeps)

    # rescale
    def rescale_Ut(y, unit, lims):
        unit = 1 if unit in ["mV", "pA"] else 1000
        xmin, xmax = lims[0] * unit, lims[1] * unit
        ymin, ymax = np.min(y), np.max(y)
        return (xmax - xmin) / (ymax - ymin) * (y - ymin) + xmin

    def rescale_t(t, dt_in, npoints):
        tN = (np.diff(t) * 1e-6).reshape(-1, 1)
        t0 = np.zeros_like(tN)
        dt = dt_in * np.ones_like(tN) * 1000
        l_t = [len(np.arange(t0_i, tN_i, dt_i)) for t0_i, tN_i, dt_i in zip(t0, tN, dt)]
        l_t = np.array(l_t).reshape(-1, 1)
        dl_t = l_t - npoints.reshape(-1, 1)
        tN -= dl_t * (dt - 1e-10)  # somehow prevents numerical errors
        trange = np.hstack([t0, tN, dt])
        return trange

    def rescale(x):
        rescaled_Ut = rescale_Ut(x.adc, x.Units, x.YLim)
        dt = np.prod(x.SampleInterval)
        rescaled_t = rescale_t(x.tim, dt, x.Npoints)
        return rescaled_t, rescaled_Ut

    # TODO: Decide how to handle signals of different length! (i.e. ramp)
    # TODO: signals -> 0 to nans according to npoints
    tqdm.pandas(desc="rescaling")
    df[["tim", "adc"]] = df.progress_apply(rescale, axis=1, result_type="expand")

    # rename cols and remove obsolete cols
    df.rename(
        columns={"adc": "Ut", "tim": "t", "DC": "dc", "Npoints": "npoints"},
        inplace=True,
    )
    df = df[["cell", "dc", "channel", "label", "npoints", "t", "Ut"]]

    # stimuli
    def vectorize_stim_inputs(*args):
        is_number = lambda x: isinstance(x, float) or isinstance(x, int)
        as_array = lambda x: np.array(x) if is_number(x) else x

        def get_reshaped_array(x):
            x = as_array(x)
            if x.ndim == 0:
                return x.reshape(1, 1)
            elif x.ndim == 1:
                return x.reshape(-1, 1)
            else:
                return x

        args = [get_reshaped_array(a) for a in args]
        dims = [x.shape[0] if x.ndim > 0 else 0 for x in args]

        for i, (a, d) in enumerate(zip(args, dims)):
            if a.shape[0] != max(dims):
                a = np.vstack([a] * max(dims))
            args[i] = a
        return args

    def I_square(t, I, t_on, t_off):
        t, I, t_on, t_off = vectorize_stim_inputs(t, I, t_on, t_off)
        is_on = np.logical_and(t_on <= t, t <= t_off).astype(float)
        return I * is_on

    def I_ramp(t, t_on=100, t_off=1100):
        t, t_on, t_off = vectorize_stim_inputs(t, t_on, t_off)

        is_on = np.logical_and(t_on <= t, t <= t_off).astype(float)
        return 25 / 1000 * (t - t_on) * is_on

    def I_sAP(t, I=2000, t_on=100, t_off=102):
        return I_square(t, I, t_on, t_off)

    # add stimulus
    def add_stimuli(U, label, npoints):
        U = U if U.ndim > 1 else U.reshape(1, -1)
        num_sweeps = U.shape[0]
        stim_types = {
            35000: (200, 1200),
            28000: (200, 1200),
            20000: (200, 800),
            25000: (100, 700),
        }
        if "ramp" in label.lower():
            stim = stim_types[npoints[0]]
            t_off_0 = (
                1200 if "Ramp" in label else 1100
            )  # capitalized stimuli start later for some reason
            t_offs = np.cumsum([t_off_0] + [500] * (num_sweeps - 1))
            stimulus = partial(I_ramp, t_on=200, t_off=t_offs)
            return stimulus

        elif "fp" in label.lower():
            stim = stim_types[npoints[0]]
            I0_idx = np.argmin(np.var(U, axis=1))
            I = np.arange(-20 * I0_idx, 20 * (num_sweeps - I0_idx), 20).reshape(-1, 1)
            stimulus = partial(I_square, I=I, t_on=stim[0], t_off=stim[1])
            return stimulus

        elif "ap" in label.lower():
            t_on = (
                200 if "AP" in label or "Cell" in label else 100
            )  # capitalized stimuli start later for some reason
            t_off = t_on + 2
            stimulus = partial(
                I_sAP, I=2000 * np.ones(num_sweeps), t_on=t_on, t_off=t_off
            )
            return stimulus

    df["It"] = df.apply(lambda x: add_stimuli(x.Ut, x.label, x.npoints), axis=1)

    return df


def get_df_filter(
    filt_df: DataFrame,
    align_by: str or Tuple[str, str],
    filter_by: str or Tuple[str, str],
    val: Any = None,
) -> Series:
    """filter one dataframe by the column of another.

    Align two dataframes by a common column (does not neccesarily have to have
    the same name, only the same data) and merge them. Then enforce that either:
    a) True if rows contain a specified value
    b) True if values in two different columns specified in filter by are equal.

    Args:
        df: Dataframe to be filtered.
        filt_df: Dataframe for filtering.
        align_by: the same or two different columns that are used to align the
            two different dataframes.
        filter_by: columns to equate. see b)
        val: value to enforce. see a)

    Returns:
        Callable that takes df and returns Series which is True where condition is met.
    """
    align_by = (align_by, align_by) if isinstance(align_by, str) else align_by
    filter_by = (filter_by, filter_by) if isinstance(filter_by, str) else filter_by

    if align_by[0] != align_by[1]:
        filt_df.rename({align_by[1]: align_by[0]}, inplace=True)

    def df_filter(df):
        i_prev = df[align_by[0]]
        df = pd.merge(df, filt_df, on=align_by[0], how="left")
        i_now = df[align_by[0]]

        assert len(i_now) == len(i_prev), "Check for duplicate entries"
        assert all(i_now == i_prev), "Incorrect alignment of indices"
        if val is None:
            return df[filter_by[0]] == df[filter_by[1]]
        else:
            return df[filter_by[0]] == val

    return df_filter


def get_std_signal_filter(trange, ut, filt_cutoff=1, notch=None):
    """convenience function to configure basic ephys signal filter."""
    dt = trange[0, -1]
    b, a = bessel(4, filt_cutoff, fs=1 / dt)
    ut = filtfilt(b, a, ut)
    if notch is not None:
        b, a = iirnotch(notch, 1.5, fs=1e3 / dt)
        ut = filtfilt(b, a, ut)
    return ut


def filter_df(
    df: DataFrame,
    filter_mask: Series or Callable,
    negate: bool = False,
    reset_index: bool = True,
) -> DataFrame:
    """Convenience function to apply filter to dataframe.

    Applies df.loc[filter_mask] or df.loc[~filter_mask]

    Args:
        df: Dataframe for filtering.
        filter_mask: filter in the form of a boolean pd.Series.
        negate: Wether to include or exclude row if filter == True.
        reset_index: Whether to reset the index after filtering.

    Returns:
        filtered dataframe.
    """
    if isinstance(filter_mask, Callable):
        filter_mask = filter_mask(df)
    reindex = lambda df: df.reset_index(drop=True) if reset_index else df
    if negate:
        return reindex(df.loc[~filter_mask])
    else:
        return reindex(df.loc[filter_mask])


def preprocess_fp_data(
    fp_data: DataFrame,
    metadata: Optional[DataFrame] = None,
    filt_cutoff: int = 1,
) -> DataFrame:
    """Apply preprocessing steps to dataframe with firing pattern data.

    Part of the ephys pipeline. Applies varies hardcoded filters to the fp data.

    Args:
        fp_data: dataframe holding the firing pattern data.
        metadata: dataframe holding the metadata.
        filt_cutoff: 4 pole bessel filter cutoff freq in kHz.

    Returns:
        preprocessed fp data.
    """

    if metadata is not None:
        # fp_data = filter_df(
        #     fp_data, get_df_filter(metadata, "cell", "Subgroup", val="Adult")
        # )
        fp_data = filter_df(
            fp_data, get_df_filter(metadata, "cell", ("channel", "firing pattern"))
        )
        fp_data = filter_df(
            fp_data,
            get_df_filter(metadata, "cell", "delete", val="Delete"),
            negate=True,
        )
        fp_data = filter_bad_sweeps(fp_data, metadata, "cell", "fp_rm_sweep")

    fp_data = filter_df(fp_data, fp_data["npoints"].apply(lambda x: len(x) > 5))

    if filt_cutoff is not None and len(fp_data) > 0:
        fp_filter = lambda df: get_std_signal_filter(df.t, df.Ut, filt_cutoff)
        fp_data["Ut"] = fp_data.apply(fp_filter, axis=1)
    return fp_data


ROOT = "../smartseq3/data/"
PATH_batch1 = ROOT + "ephys_batch1/Ephys Mat Data"
PATH_batch2 = ROOT + "ephys_batch2/Ephys Mat Data"

# metadata
metadata_batch1 = pd.read_excel(
    ROOT + "ephys_batch1/cell_metadata_batch1.xlsx", dtype={"fp_rm_sweep": str}
)
metadata_batch2 = pd.read_excel(
    ROOT + "ephys_batch2/cell_metadata_batch2.xlsx", dtype={"fp_rm_sweep": str}
)
cols = [
    "cell",
    "delete",
    "delete reason",
    "single AP",
    "RAMP_AP",
    "firing pattern",
    "fp_rm_sweep",
]
metadata = pd.concat([metadata_batch1[cols + ["Subgroup"]], metadata_batch2[cols]])

# file paths
fpaths_batch1 = [
    os.path.join(PATH_batch1, f) for f in os.listdir(PATH_batch1) if ".mat" in f
]
fpaths_batch2 = [
    os.path.join(PATH_batch2, f) for f in os.listdir(PATH_batch2) if ".mat" in f
]
fpaths = fpaths_batch1 + fpaths_batch2


data = import_mats(fpaths[:100])
fp_data = filter_df(data, get_regex_filter("label", "fp"))
fp_data = preprocess_fp_data(fp_data, metadata, 5)

rescaling: 100%|██████████| 341/341 [00:01<00:00, 186.00it/s]


In [43]:
t_set, u_set, i_set, dc = fp_data[["t", "Ut", "It", "dc"]].iloc[0]
t_set = trange2t(t_set)
i_set = i_set(t_set)
t_set *= 1e-3

# create sweepset
test_sweepset = EphysSweepSet(
    t_set,
    u_set,
    i_set,
    filter=5,
    dc_offset=dc,
)

test_sweepset.add_features(available_spike_features())
test_sweepset.add_features(available_sweepset_features())
test_sweepset.get_features()

AssertionError: data must be a EphysSweepSetFeatureExtractor object

In [38]:
from ephyspy.plot import plot_spike_features

In [39]:
test_sweepset[20].plot_features()

TypeError: plot_features() missing 1 required positional argument: 'fts'

In [3]:
# dcts = []
# for i in tqdm(range(250)):
#     t_set, u_set, i_set, dc = fp_data[["t", "Ut", "It", "dc"]].iloc[i]
#     t_set = trange2t(t_set)
#     i_set = i_set(t_set)
#     t_set *= 1e-3
#     start, end = t_set[0, 0], t_set[0, -1]

#     # create sweepset
#     test_sweepset = EphysSweepSetFeatureExtractor(
#         t_set, u_set, i_set, filter=5, dc_offset=dc
#     )

#     # add available spike, sweep and sweepset features to sweepsetextractor
#     test_sweepset.set_stimulus_amplitude_calculator(get_sweep_stim_amp)
#     for ft, ft_func in get_available_spike_features().items():
#         test_sweepset.add_spike_feature(ft, ft_func)
#     for ft, ft_func in get_available_sweep_features(return_ft_info=False).items():
#         test_sweepset.add_sweep_feature(ft, ft_func)
#     for ft, ft_func in get_available_sweepset_features(return_ft_info=False).items():
#         test_sweepset.add_sweepset_feature(ft, ft_func)

#     # process sweepset features
#     test_sweepset.process()

#     dcts.append(test_sweepset.get_sweepset_features())
# df = pd.DataFrame(dcts)

In [42]:
for i, (t_set, u_set, i_set, dc) in tqdm(fp_data[["t", "Ut", "It", "dc"]].iterrows()):
    t_set = trange2t(t_set)
    i_set = i_set(t_set)
    t_set *= 1e-3

    # create sweepset
    test_sweepset = EphysSweepSetFeatureExtractor(
        t_set,
        u_set,
        i_set,
        filter=5,
        dc_offset=dc,
    )

    test_sweepset.add_features(available_spike_features())
    test_sweepset.add_features(available_sweepset_features())
    test_sweepset.get_features()

89it [15:33, 10.48s/it]


In [18]:
# measure execution time
import time

t_set, u_set, i_set, dc = fp_data[["t", "Ut", "It", "dc"]].iloc[13]
t_set = trange2t(t_set)
i_set = i_set(t_set)
t_set *= 1e-3
start, end = t_set[0, 0], t_set[0, -1]

# create sweepset
test_sweepset = EphysSweepSetFeatureExtractor(
    t_set,
    u_set,
    i_set,
    filter=5,
    dc_offset=dc,
)
test_sweepset.add_features(available_spike_features())
test_sweepset.add_features(available_sweepset_features())

test_sweep = test_sweepset[11]

times = {"time": [], "feature": [], "lookup": []}

n = len(test_sweepset)
for ft_name, FT in available_sweep_features().items():
    if ft_name in available_sweepset_features():
        SweepFT = available_sweepset_features()[ft_name]
        t0 = time.time()
        FT()(test_sweep)
        t1 = time.time()
        t2 = time.time()
        FT()(test_sweep)
        t3 = time.time()

        test_sweep.features = {}

        t3 = time.time()
        SweepFT(test_sweepset)
        t4 = time.time()

        t5 = time.time()
        SweepFT(test_sweepset)
        t6 = time.time()

        t1_nolook = t1 - t0
        t1_look = t3 - t2
        tn_nolook = t4 - t3
        tn_look = t6 - t5

        # print(f"1x w.o. lookup vs 1x w. lookup: {t1_nolook:.5f} vs {t1_look:.5f}")
        # print(
        #     f"sweepset w.o. lookup vs sweepset w. lookup: {tn_nolook:.5f} vs {tn_look:.5f}"
        # )
        times["feature"] += 2 * [ft_name]
        times["time"] += [t1_nolook, t1_look]
        times["lookup"] += [False, True]

        # print(f"w.o. lookup {n}x vs sweepset: {n * t1_nolook:.5f} vs {tn_nolook:.5f}")
        # print(f"w. lookup {n}x vs sweepset: {n * t1_look:.5f} vs {tn_look:.5f}")

        # assert t1_nolook > t1_look, f"1x w.o. lookup > 1x w. lookup"
        # assert tn_nolook > tn_look, f"sweepset w.o. lookup > sweepset w. lookup"

        # assert (
        #     abs(n * t1_nolook - tn_nolook) / tn_nolook < 0.5
        # ), f"{n}x w.o. lookup > sweepset w.o. lookup"
        # assert (
        #     abs(n * t1_look - tn_look) / tn_look < 0.5
        # ), f"{n}x w. lookup > sweepset w. lookup"

times = pd.DataFrame(times)

AssertionError: data must be a EphysSweepSetFeatureExtractor object

In [ ]:
times.sort_values(by="time", ascending=False)

In [ ]:
sns.histplot(data=times, x="time", hue="lookup")
plt.show()

In [ ]:
for FT in fetch_available_fts():
    ft_name = FT.__name__.lower()
    if not (any(w in ft_name for w in ["sweepset", "apfeature", "rheobase", "dfdi"])):
        ft = FT(test_sweep)
        print(ft)

In [ ]:
# class recomputed_features:
#     def __init__(self, feature, recompute=True):
#         self.feature = feature
#         self.recompute = recompute
#         self.without_recompute = self.feature.lookup_sweep_feature

#     def __enter__(self):
#         def toggle_recompute(func):
#             def wrapped_func(*args, **kwargs):
#                 kwargs["recompute"] = self.recompute
#                 return func(*args, **kwargs)
#             return wrapped_func


#         self.feature.data.process_spikes()
#         self.feature.lookup_sweep_feature = toggle_recompute(
#             self.feature.lookup_sweep_feature
#         )

#         return self.feature

#     def __exit__(self, exc_type, exc_value, traceback):
#         self.feature.lookup_sweep_feature = self.without_recompute

In [ ]:
t_set, u_set, i_set, dc = fp_data[["t", "Ut", "It", "dc"]].iloc[248]
t_set = trange2t(t_set)
i_set = i_set(t_set)
t_set *= 1e-3
start, end = t_set[0, 0], t_set[0, -1]

# create sweepset
test_sweepset = EphysSweepSetFeatureExtractor(
    t_set, u_set, i_set, filter=5, dc_offset=dc
)

# add available spike, sweep and sweepset features to sweepsetextractor
test_sweepset.set_stimulus_amplitude_calculator(get_sweep_stim_amp)
for ft, ft_func in get_available_spike_features().items():
    test_sweepset.add_spike_feature(ft, ft_func)
for ft, ft_func in get_available_sweep_features(return_ft_info=False).items():
    test_sweepset.add_sweep_feature(ft, ft_func)
for ft, ft_func in get_available_sweepset_features(return_ft_info=False).items():
    test_sweepset.add_sweepset_feature(ft, ft_func)

# process sweepset features
test_sweepset.process()

In [ ]:
df.hist(figsize=(12, 12))
plt.tight_layout()
plt.show()

In [ ]:
# cell.process("long_squares")  # needs to be run before spikes
# cell.process("long_squares_spiking")


In [ ]:
# cell.long_squares_features("spiking").sweep_features("avg_rate")

In [ ]:
# ft_df = fp_sweepset.get_sweep_features()  # .applymap(strip_info)
# sweepset_fts = {
#     ft: ft_func(ft_df) for ft, ft_func in ef.get_fp_sweepset_ft_dict().items()
# }
# sweepset_fts